In [3]:
pip install PyMuPDF pandas

  Obtaining dependency information for PyMuPDF from https://files.pythonhosted.org/packages/71/c2/a9059607f80dcaf2392f991748cfc53456820392c0220cff02572653512a/pymupdf-1.25.5-cp39-abi3-win_amd64.whl.metadata
   ---------------------------------------- 0.0/16.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/16.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/16.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/16.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/16.6 MB 245.8 kB/s eta 0:01:08
   ---------------------------------------- 0.1/16.6 MB 595.3 kB/s eta 0:00:28
    --------------------------------------- 0.2/16.6 MB 935.2 kB/s eta 0:00:18
    --------------------------------------- 0.3/16.6 MB 1.3 MB/s eta 0:00:13
   - -------------------------------------- 0.5/16.6 MB 1.6 MB/s eta 0:00:10
   - -------------------------------------- 0.7/16.6 MB 1.9 MB/s eta 0:00:09
   -- ------------------------------------- 0

In [1]:
# Module 1: Read and Print Resume PDF
import fitz  # PyMuPDF
from pathlib import Path

# Define the path to your resume PDF
pdf_path = "TE_Resume.pdf"  # <- Change this to your actual PDF path

# Open the PDF
doc = fitz.open(pdf_path)

# Extract and print text from all pages
for page_num, page in enumerate(doc, start=1):
    resume_text = page.get_text()
    text = resume_text
    print(f"\n--- Page {page_num} ---\n")
    print(resume_text)


--- Page 1 ---

NIKHIL .M. SUTAR
SUMMARY
Innovative developer with expertise in C, Java, Python, and JavaScript. Proficient in utilizing frameworks such
as Flask, Django, and Bootstrap, with ongoing learning in React.js. Experienced in version control with Git and
GitHub and adept at deploying projects on Vercel. Committed to leveraging AI tools, including ChatGPT, to
enhance project outcomes. Seeking a dynamic role to apply technical skills and drive project success.
PROJECTS
EDUCATION
Live Link : https://success-classes-of-engineering.vercel.app/
Frontend : HTML , CSS 
Backend : Flask , Python
2. Success Classes of Engineering
    Jun 2024  - Jun 2024
ADDITIONAL INFORMATION
Another Skills Certification :  PC Operation (Excel, PowerPoint, Word), Power BI
Languages: English, Hindi , Marathi
Hobbies: Drawing Architectural structures 
4. Portfolio Website
Live Link : https://carpentrycoder.github.io/Resume/
Frontend: HTML , JS , Tailwind CSS
Jan 2025 - Jan 2025
Github | +91-7738544966 |

In [10]:
import re

contact_info = resume_df["text"].str.contains(r"\+91|\@|gmail\.com|LinkedIn|GitHub", case=False)
print(resume_df[contact_info]["text"])

4     GitHub and adept at deploying projects on Verc...
23             https://carpentrycoder.github.io/Resume/
26                                         Github | +91
28     7738544966 | nikhilsutar621@gmail.com | LinkedIn
44                                        LinkedIN Post
Name: text, dtype: object


In [1]:
import fitz  # PyMuPDF
import re
from pathlib import Path
import pprint

# Read PDF
pdf_path = "TE_Resume.pdf"
doc = fitz.open(pdf_path)

# Extract full text
resume_text = "\n".join(page.get_text() for page in doc)

# ---------- STRUCTURED PARSING ----------
structured_resume = {}

# Name (assume first non-empty line)
lines = resume_text.splitlines()
structured_resume["name"] = next((line.strip() for line in lines if line.strip()), "Unknown Name")

# Summary
summary_match = re.search(r"SUMMARY\n(.*?)\nPROJECTS", resume_text, re.DOTALL)
structured_resume["summary"] = summary_match.group(1).strip() if summary_match else ""

# Projects
projects_section = re.search(r"PROJECTS\n(.*?)\nEDUCATION", resume_text, re.DOTALL)
projects = []
if projects_section:
    raw = projects_section.group(1).strip()
    project_entries = re.split(r"\n\d+\.\s", raw)
    for entry in project_entries[1:]:
        lines = entry.strip().splitlines()
        title = lines[0] if lines else "Untitled"
        details = "\n".join(lines[1:]) if len(lines) > 1 else ""
        projects.append({"title": title, "details": details})
structured_resume["projects"] = projects

# Education
edu_match = re.search(r"EDUCATION\n(.*?)\nSKILLS", resume_text, re.DOTALL)
structured_resume["education"] = edu_match.group(1).strip() if edu_match else ""

# Skills
skills_match = re.search(r"SKILLS\s+(.*?)\nADDITIONAL INFORMATION", resume_text, re.DOTALL)
skills_text = skills_match.group(1).strip() if skills_match else ""
skills = {}
for line in skills_text.splitlines():
    if ":" in line:
        key, value = line.split(":", 1)
        skills[key.strip()] = [v.strip() for v in value.split(",")]
structured_resume["skills"] = skills

# Additional Info
add_info_match = re.search(r"ADDITIONAL INFORMATION\n(.*?)(Github|\+91|Email|LinkedIn)", resume_text, re.DOTALL)
additional_text = add_info_match.group(1).strip() if add_info_match else ""
additional = {}
for line in additional_text.splitlines():
    if ":" in line:
        key, value = line.split(":", 1)
        additional[key.strip()] = [v.strip() for v in value.split(",")]
structured_resume["additional_info"] = additional

# Contact info: grab line with all 4 parts
contact_line = next((line for line in lines if "Github" in line and "|" in line), "")
contact_parts = [part.strip() for part in contact_line.split("|")]
contact_keys = ["github", "phone", "email", "linkedin"]
structured_resume["contact"] = {k: contact_parts[i] if i < len(contact_parts) else "N/A"
                                 for i, k in enumerate(contact_keys)}

# ---------- Print Final Output ----------
print("\n✅ Final Structured Resume Data:\n")
pprint.pprint(structured_resume, width=150)


✅ Final Structured Resume Data:

{'additional_info': {'Another Skills Certification': ['PC Operation (Excel', 'PowerPoint', 'Word)', 'Power BI'],
                     'Frontend': ['HTML', 'JS', 'Tailwind CSS'],
                     'Hobbies': ['Drawing Architectural structures'],
                     'Languages': ['English', 'Hindi', 'Marathi'],
                     'Live Link': ['https://carpentrycoder.github.io/Resume/']},
 'contact': {'email': 'nikhilsutar621@gmail.com', 'github': 'Github', 'linkedin': 'LinkedIn', 'phone': '+91-7738544966'},
 'education': 'Live Link : https://success-classes-of-engineering.vercel.app/\n'
              'Frontend : HTML , CSS \n'
              'Backend : Flask , Python\n'
              '2. Success Classes of Engineering\n'
              '    Jun 2024  - Jun 2024\n'
              'ADDITIONAL INFORMATION\n'
              'Another Skills Certification :  PC Operation (Excel, PowerPoint, Word), Power BI\n'
              'Languages: English, Hindi , Marat

In [2]:
import fitz  # PyMuPDF

def extract_text_from_pdf(file_path):
    doc = fitz.open(file_path)
    text = ""
    for page in doc:
        text += f"\n--- Page {page.number + 1} ---\n"
        text += page.get_text()
    return text

# 📂 Replace this with your resume file path
pdf_path = "TE_Resume.pdf"
resume_text = extract_text_from_pdf(pdf_path)
print(resume_text[:5000])  # printing only first 1000 chars


--- Page 1 ---
NIKHIL .M. SUTAR
SUMMARY
Innovative developer with expertise in C, Java, Python, and JavaScript. Proficient in utilizing frameworks such
as Flask, Django, and Bootstrap, with ongoing learning in React.js. Experienced in version control with Git and
GitHub and adept at deploying projects on Vercel. Committed to leveraging AI tools, including ChatGPT, to
enhance project outcomes. Seeking a dynamic role to apply technical skills and drive project success.
PROJECTS
EDUCATION
Live Link : https://success-classes-of-engineering.vercel.app/
Frontend : HTML , CSS 
Backend : Flask , Python
2. Success Classes of Engineering
    Jun 2024  - Jun 2024
ADDITIONAL INFORMATION
Another Skills Certification :  PC Operation (Excel, PowerPoint, Word), Power BI
Languages: English, Hindi , Marathi
Hobbies: Drawing Architectural structures 
4. Portfolio Website
Live Link : https://carpentrycoder.github.io/Resume/
Frontend: HTML , JS , Tailwind CSS
Jan 2025 - Jan 2025
Github | +91-7738544966 | 

In [3]:
import re

def clean_text(text):
    text = re.sub(r'\n+', '\n', text)  # remove extra line breaks
    text = re.sub(r'\s{2,}', ' ', text)  # remove extra spaces
    return text.strip()

cleaned_resume = clean_text(resume_text)
print(cleaned_resume)

--- Page 1 ---
NIKHIL .M. SUTAR
SUMMARY
Innovative developer with expertise in C, Java, Python, and JavaScript. Proficient in utilizing frameworks such
as Flask, Django, and Bootstrap, with ongoing learning in React.js. Experienced in version control with Git and
GitHub and adept at deploying projects on Vercel. Committed to leveraging AI tools, including ChatGPT, to
enhance project outcomes. Seeking a dynamic role to apply technical skills and drive project success.
PROJECTS
EDUCATION
Live Link : https://success-classes-of-engineering.vercel.app/
Frontend : HTML , CSS Backend : Flask , Python
2. Success Classes of Engineering Jun 2024 - Jun 2024
ADDITIONAL INFORMATION
Another Skills Certification : PC Operation (Excel, PowerPoint, Word), Power BI
Languages: English, Hindi , Marathi
Hobbies: Drawing Architectural structures 4. Portfolio Website
Live Link : https://carpentrycoder.github.io/Resume/
Frontend: HTML , JS , Tailwind CSS
Jan 2025 - Jan 2025
Github | +91-7738544966 | nikhilsut

In [6]:
import re

# ✅ Expanded Skills Database (you can keep updating this!)
SKILLS_DB = [
    'C', 'C++', 'Java', 'Python', 'JavaScript', 'TypeScript', 'HTML', 'CSS', 'SQL',
    'Bootstrap', 'Tailwind CSS', 'React.js', 'Vue.js', 'Angular',
    'Flask', 'Django', 'Node.js', 'Express.js',
    'Pandas', 'NumPy', 'Matplotlib', 'Seaborn', 'Plotly',
    'Scikit-Learn', 'Machine Learning', 'Deep Learning', 'LLMs',
    'Hugging Face', 'Transformers', 'TensorFlow', 'PyTorch',
    'OpenCV', 'Power BI', 'Excel', 'Git', 'GitHub', 'Docker', 'PostgreSQL', 'MongoDB'
]

def extract_email(text):
    match = re.search(r'[\w\.-]+@[\w\.-]+', text)
    return match.group(0) if match else None

def extract_phone(text):
    match = re.search(r'\+91[-\s]?[0-9]{10}|\b[7-9]\d{9}\b', text)
    return match.group(0) if match else None

def extract_name(text):
    lines = text.split("\n")
    for line in lines:
        if "SUMMARY" in line.upper():
            return lines[lines.index(line) - 1].strip()
    return "Name not found"

def extract_skills(text):
    text = text.lower()
    found_skills = []
    for skill in SKILLS_DB:
        if skill.lower() in text:
            found_skills.append(skill)
    return list(set(found_skills))  # Unique skills only

# --- Assuming 'cleaned_resume' and 'resume_text' are defined already ---
email = extract_email(cleaned_resume)
phone = extract_phone(cleaned_resume)
name = extract_name(resume_text)
skills = extract_skills(cleaned_resume)

# 🖨️ Output Results
print("Name:", name)
print("Email:", email)
print("Phone:", phone)
print(f"Skills ({len(skills)} matched):", ', '.join(skills))

Name: NIKHIL .M. SUTAR
Email: nikhilsutar621@gmail.com
Phone: +91-7738544966
Skills (27 matched): C, Machine Learning, Bootstrap, Excel, SQL, Tailwind CSS, Scikit-Learn, Git, HTML, Pandas, Django, Python, NumPy, Flask, Hugging Face, React.js, Seaborn, Node.js, Matplotlib, JavaScript, PostgreSQL, GitHub, Java, Transformers, LLMs, CSS, Power BI


In [8]:
import re

# Sample predefined skill set
SKILLS_DB = [
    'C', 'C++', 'Java', 'Python', 'JavaScript', 'HTML', 'CSS', 'SQL',
    'Bootstrap', 'React.js', 'Flask', 'Django', 'Node.js',
    'Pandas', 'NumPy', 'Matplotlib', 'Seaborn',
    'Scikit-Learn', 'Machine Learning', 'LLMs',
    'Hugging Face', 'Transformers', 'Power BI',
    'Excel', 'Git', 'GitHub', 'PostgreSQL', 'Tailwind CSS'
]

def extract_email(text):
    match = re.search(r'[\w\.-]+@[\w\.-]+', text)
    return match.group(0) if match else None

def extract_phone(text):
    match = re.search(r'\+91[-\s]?[0-9]{10}|\b[7-9]\d{9}\b', text)
    return match.group(0) if match else None

def extract_name(text):
    lines = text.split("\n")
    for line in lines:
        if "SUMMARY" in line.upper():
            return lines[lines.index(line) - 1].strip()
    return "Name not found"

def extract_skills(text):
    text = text.lower()
    found_skills = [skill for skill in SKILLS_DB if skill.lower() in text]
    return list(set(found_skills))

def extract_education(text):
    education_keywords = ['university', 'college', 'bachelor', 'master', 'b.tech', 'm.tech', 'cgpa', 'degree']
    lines = text.lower().split("\n")
    education = [line.strip() for line in lines if any(keyword in line for keyword in education_keywords)]
    return education if education else ["Education info not found"]

def extract_experience(text):
    experience_keywords = ['project', 'experience', 'internship', 'jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
    lines = text.lower().split("\n")
    experience = [line.strip() for line in lines if any(keyword in line for keyword in experience_keywords)]
    return experience if experience else ["Experience info not found"]

# Run the extractors
email = extract_email(cleaned_resume)
phone = extract_phone(cleaned_resume)
name = extract_name(resume_text)
skills = extract_skills(cleaned_resume)
education = extract_education(cleaned_resume)
experience = extract_experience(cleaned_resume)

# Print the results
print("Name:", name)
print("Email:", email)
print("Phone:", phone)
print(f"Skills ({len(skills)} matched):", ', '.join(skills))
print("\nEducation Section:")
for edu in education:
    print("-", edu)

print("\nExperience / Projects Section:")
for exp in experience:
    print("-", exp)

Name: NIKHIL .M. SUTAR
Email: nikhilsutar621@gmail.com
Phone: +91-7738544966
Skills (27 matched): C, Machine Learning, Bootstrap, Excel, SQL, Scikit-Learn, Tailwind CSS, Git, HTML, Pandas, Django, Python, NumPy, Flask, Hugging Face, React.js, Seaborn, Node.js, Matplotlib, JavaScript, PostgreSQL, GitHub, Java, Transformers, LLMs, CSS, Power BI

Education Section:
- konkan gyanpeeth college of engineering , affiliated to the university of mumbai artificial intelligence and data science | cgpa of 5 sems: 6.70 2023 - 2026 (expected)

Experience / Projects Section:
- summary
- innovative developer with expertise in c, java, python, and javascript. proficient in utilizing frameworks such
- as flask, django, and bootstrap, with ongoing learning in react.js. experienced in version control with git and
- github and adept at deploying projects on vercel. committed to leveraging ai tools, including chatgpt, to
- enhance project outcomes. seeking a dynamic role to apply technical skills and drive 

In [15]:
import re
import json

SKILLS_DB = [
    'C', 'C++', 'Java', 'Python', 'JavaScript', 'HTML', 'CSS', 'SQL', 'Tailwind CSS',
    'Bootstrap', 'React.js', 'Flask', 'Django', 'Node.js', 'Pandas', 'NumPy',
    'Matplotlib', 'Seaborn', 'Scikit-Learn', 'Machine Learning', 'LLMs',
    'Hugging Face', 'Transformers', 'Power BI', 'PostgreSQL', 'Git', 'GitHub', 'Excel'
]

def extract_email(text):
    match = re.search(r'[\w\.-]+@[\w\.-]+', text)
    return match.group(0) if match else None

def extract_phone(text):
    match = re.search(r'\+91[-\s]?[0-9]{10}|\b[7-9]\d{9}\b', text)
    return match.group(0) if match else None

def extract_name(text):
    lines = text.split("\n")
    for line in lines:
        if "SUMMARY" in line.upper():
            return lines[lines.index(line) - 1].strip()
    return lines[0].strip()  # fallback

def extract_skills(text):
    text = text.lower()
    found_skills = []
    for skill in SKILLS_DB:
        if skill.lower() in text:
            found_skills.append(skill)
    return list(set(found_skills))

def extract_education(text):
    lines = text.lower().splitlines()
    edu_lines = [line.strip() for line in lines if 'university' in line or 'college' in line or 'cgpa' in line]
    return edu_lines

def extract_experience_and_projects(text):
    lines = text.lower().splitlines()
    exp_lines = []
    for line in lines:
        if any(keyword in line for keyword in ['summary', 'projects', 'jun', 'jan', 'link']):
            exp_lines.append(line.strip())
    return exp_lines

def extract_projects(text):
    lines = text.splitlines()
    projects = []
    current_project = {}

    for line in lines:
        lower_line = line.lower()

        # New project start
        if "post" in line.lower():
            if current_project:
                projects.append(current_project)
                current_project = {}
            current_project['title'] = line.strip()

        # Tech stack detection from SKILLS_DB
        for skill in SKILLS_DB:
            if skill.lower() in lower_line:
                current_project.setdefault('tech', set()).add(skill)

        # Link extraction
        if "live link" in lower_line:
            current_project['live link'] = line.strip().split(":", 1)[-1].strip()

    if current_project:
        # Final append after last project
        if 'tech' in current_project:
            current_project['tech'] = list(current_project['tech'])
        projects.append(current_project)

    # Convert sets to lists
    for project in projects:
        if 'tech' in project:
            project['tech'] = list(project['tech'])

    return projects



# Extract info
email = extract_email(text)
phone = extract_phone(text)
name = extract_name(text)
skills = extract_skills(text)
education = extract_education(text)
experience = extract_experience_and_projects(text)
projects = extract_projects(text)

# Display the results
print("📛 Name:", name)
print("📧 Email:", email)
print("📞 Phone:", phone)
print("🧠 Skills Matched:", len(skills))
print("🧩 Skills:", skills)
print("\n🎓 Education:")
for edu in education:
    print("-", edu)

print("\n💼 Experience & Project Highlights:")
for exp in experience:
    print("-", exp)

print("\n📂 Projects:")
for project in projects:
    title = project.get("title", "Untitled Project")
    print(f"\n🔹 {title}")
    
    tech = project.get("tech")
    if tech:
        print(f"   🛠️ Tech Stack: {', '.join(tech)}")
    
    live_link = project.get("live link") or project.get("live_link")
    if live_link:
        print(f"   🌐 Live Link: {live_link}")



📛 Name: NIKHIL .M. SUTAR
📧 Email: nikhilsutar621@gmail.com
📞 Phone: +91-7738544966
🧠 Skills Matched: 27
🧩 Skills: ['C', 'Machine Learning', 'Bootstrap', 'Excel', 'SQL', 'Tailwind CSS', 'Scikit-Learn', 'Git', 'HTML', 'Pandas', 'Django', 'Python', 'NumPy', 'Flask', 'Hugging Face', 'React.js', 'Seaborn', 'Node.js', 'Matplotlib', 'JavaScript', 'PostgreSQL', 'GitHub', 'Java', 'Transformers', 'LLMs', 'CSS', 'Power BI']

🎓 Education:
- konkan gyanpeeth college of engineering , affiliated to the university of mumbai
- artificial intelligence and data science | cgpa of 5 sems: 6.70

💼 Experience & Project Highlights:
- summary
- as flask, django, and bootstrap, with ongoing learning in react.js. experienced in version control with git and
- github and adept at deploying projects on vercel. committed to leveraging ai tools, including chatgpt, to
- projects
- live link : https://success-classes-of-engineering.vercel.app/
- jun 2024  - jun 2024
- live link : https://carpentrycoder.github.io/resume

In [17]:
!pip install pdfplumber python-docx

  Obtaining dependency information for pdfplumber from https://files.pythonhosted.org/packages/e6/c4/d2e09fbc937d1f76baae34e526662cc718e23a904321bf4a40282d190033/pdfplumber-0.11.6-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/42.8 kB ? eta -:--:--
     --------------------------- ---------- 30.7/42.8 kB 660.6 kB/s eta 0:00:01
     -------------------------------------- 42.8/42.8 kB 524.4 kB/s eta 0:00:00
  Obtaining dependency information for python-docx from https://files.pythonhosted.org/packages/3e/3d/330d9efbdb816d3f60bf2ad92f05e1708e4a1b9abe80461ac3444c83f749/python_docx-1.1.2-py3-none-any.whl.metadata
  Obtaining dependency information for pdfminer.six==20250327 from https://files.pythonhosted.org/packages/29/2f/409e174b5a0195aa6a814c7359a1285f1c887a4c84aff17ed03f607c06ba/pdfminer_six-20250327-py3-none-any.whl.metadata
  Obtaining dependency information for pypdfium2>=4.18.0 from https://files.pythonhosted.org/packages/a4/f3/8d3a350efb4286b5ebdabcf67

In [18]:
import re
import json
import pdfplumber
import docx

# SKILLS_DB and your extract_ functions defined here...

def read_pdf(file_path):
    with pdfplumber.open(file_path) as pdf:
        return "\n".join(page.extract_text() for page in pdf.pages if page.extract_text())

def read_docx(file_path):
    doc = docx.Document(file_path)
    return "\n".join(para.text for para in doc.paragraphs)

def read_txt(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return f.read()

def extract_resume_data(file_path):
    if file_path.endswith(".pdf"):
        text = read_pdf(file_path)
    elif file_path.endswith(".docx"):
        text = read_docx(file_path)
    elif file_path.endswith(".txt"):
        text = read_txt(file_path)
    else:
        raise ValueError("Unsupported file format")

    # Extract info
    email = extract_email(text)
    phone = extract_phone(text)
    name = extract_name(text)
    skills = extract_skills(text)
    education = extract_education(text)
    experience = extract_experience_and_projects(text)
    projects = extract_projects(text)

    return {
        "name": name,
        "email": email,
        "phone": phone,
        "skills": skills,
        "education": education,
        "experience": experience,
        "projects": projects
    }

# Usage
resume_info = extract_resume_data("TE_Resume.pdf")
print(json.dumps(resume_info, indent=4))


{
    "name": "Github | +91-7738544966 | nikhilsutar621@gmail.com | LinkedIn",
    "email": "nikhilsutar621@gmail.com",
    "phone": "+91-7738544966",
    "skills": [
        "C",
        "Machine Learning",
        "Bootstrap",
        "Excel",
        "SQL",
        "Tailwind CSS",
        "Scikit-Learn",
        "Git",
        "HTML",
        "Pandas",
        "Django",
        "Python",
        "NumPy",
        "Flask",
        "Hugging Face",
        "React.js",
        "Seaborn",
        "Node.js",
        "Matplotlib",
        "JavaScript",
        "PostgreSQL",
        "GitHub",
        "Java",
        "Transformers",
        "LLMs",
        "CSS",
        "Power BI"
    ],
    "education": [
        "konkan gyanpeeth college of engineering , affiliated to the university of mumbai",
        "artificial intelligence and data science | cgpa of 5 sems: 6.70 2023 - 2026 (expected)"
    ],
    "experience": [
        "github | +91-7738544966 | nikhilsutar621@gmail.com | linkedin",
 

In [1]:
import csv

# Sample data (you can append more dictionaries like this to the list)
resume_data = [
    {
        "Name": "NIKHIL M. SUTAR",
        "Email": "nikhilsutar621@gmail.com",
        "Phone": "+91-7738544966",
        "Education": "konkan gyanpeeth college of engineering , affiliated to the university of mumbai; artificial intelligence and data science | cgpa of 5 sems: 6.70",
        "Skills": ['Python', 'Java', 'C', 'JavaScript', 'Django', 'Flask', 'React', 'Machine Learning', 'LLM', 'SQL',
                   'Pandas', 'Numpy', 'Seaborn', 'Bootstrap', 'GitHub', 'Tailwind', 'HTML', 'CSS', 'Tailwind CSS',
                   'React.js', 'Node.js', 'NumPy', 'Matplotlib', 'Scikit-Learn', 'LLMs', 'Hugging Face',
                   'Transformers', 'Power BI', 'Excel', 'Git', 'PostgreSQL']
    },
    # Add more entries here...
]

# CSV file name
filename = "parsed_resumes.csv"

# Write to CSV
with open(filename, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=["Name", "Email", "Phone", "Education", "Skills"])
    writer.writeheader()
    for entry in resume_data:
        entry['Skills'] = ', '.join(entry['Skills'])  # convert list to string
        writer.writerow(entry)

print(f"✅ CSV file '{filename}' created successfully!")

✅ CSV file 'parsed_resumes.csv' created successfully!


In [2]:
!pip install faker

   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ---------- ----------------------------- 0.5/1.9 MB 3.4 MB/s eta 0:00:01
   --------------------- ------------------ 1.0/1.9 MB 3.6 MB/s eta 0:00:01
   -------------------------------------- - 1.8/1.9 MB 3.4 MB/s eta 0:00:01
   ---------------------------------------- 1.9/1.9 MB 3.2 MB/s eta 0:00:00


In [3]:
import csv
import random
from faker import Faker

fake = Faker()

# Define skill pool
skill_pool = [
    'Python', 'Java', 'C', 'C++', 'JavaScript', 'TypeScript', 'HTML', 'CSS',
    'SQL', 'Bootstrap', 'Tailwind CSS', 'React.js', 'Vue.js', 'Angular',
    'Flask', 'Django', 'Node.js', 'Express.js',
    'Pandas', 'NumPy', 'Matplotlib', 'Seaborn', 'Plotly',
    'Scikit-Learn', 'Machine Learning', 'Deep Learning', 'LLMs',
    'Hugging Face', 'Transformers', 'TensorFlow', 'PyTorch',
    'OpenCV', 'Power BI', 'Excel', 'Git', 'GitHub', 'Docker', 'PostgreSQL', 'MongoDB'
]

# College and course sample data
colleges = [
    "IIT Bombay", "VJTI Mumbai", "COEP Pune", "BITS Pilani", "MIT Manipal", 
    "Konkan Gyanpeeth College of Engineering", "Thakur College", "Sardar Patel Institute"
]
courses = [
    "Artificial Intelligence and Data Science", "Computer Engineering",
    "Information Technology", "Electronics and Telecommunication", "Data Science"
]

# Generate multiple fake resume entries
def generate_fake_resumes(n=10):
    resumes = []
    for _ in range(n):
        name = fake.name()
        email = fake.email()
        phone = fake.phone_number()
        college = random.choice(colleges)
        course = random.choice(courses)
        cgpa = round(random.uniform(6.0, 9.5), 2)

        education = f"{college}; {course} | CGPA: {cgpa}"

        skills = random.sample(skill_pool, k=random.randint(8, 15))  # Random 8-15 skills

        resumes.append({
            "Name": name,
            "Email": email,
            "Phone": phone,
            "Education": education,
            "Skills": skills
        })
    return resumes

# CSV file name
filename = "parsed_fake_resumes.csv"

# Generate data
resume_data = generate_fake_resumes(20)  # change number for more/less candidates

# Write to CSV
with open(filename, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=["Name", "Email", "Phone", "Education", "Skills"])
    writer.writeheader()
    for entry in resume_data:
        entry['Skills'] = ', '.join(entry['Skills'])  # convert list to string
        writer.writerow(entry)

print(f"✅ CSV file '{filename}' with {len(resume_data)} fake resumes created successfully!")

✅ CSV file 'parsed_fake_resumes.csv' with 20 fake resumes created successfully!
